In [ ]:
import numpy as np
import pandas as pd
import time
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, BaggingRegressor, ExtraTreesRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.svm import SVR
from mlxtend.regressor import StackingCVRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import ElasticNet, Lasso, BayesianRidge, LassoLarsIC
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from scipy.stats import skew, norm
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold, KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder
from category_encoders import MEstimateEncoder
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.datasets import make_regression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from hyperopt import hp, tpe, Trials, STATUS_OK
from hyperopt.fmin import fmin
import warnings
warnings.filterwarnings(action='ignore')
import textwrap
def wrap_labels(ax, width, break_long_words=False):
    labels = []
    for label in ax.get_xticklabels():
        text = label.get_text()
        labels.append(textwrap.fill(text, width=width,
                      break_long_words=break_long_words))
    ax.set_xticklabels(labels, rotation=0)

In [ ]:
train = pd.read_csv(sep='|')
test = pd.read_csv(sep='|')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train = train.applymap(lambda x: x.lower() if isinstance(x,str) else x)
test = test.applymap(lambda x: x.lower() if isinstance(x,str) else x)

In [ ]:
info = pd.concat([train.dtypes, train.nunique(), train.isnull().sum()*100/len(train)], axis=1)
info.columns = ['dtypes', 'nunique', 'nan %']
info

In [ ]:
info = pd.concat([test.dtypes, test.nunique(), test.isnull().sum()*100/len(test)], axis=1)
info.columns = ['dtypes', 'nunique', 'nan %']
info

In [ ]:
train.drop('id', axis=1, inplace=True)

In [ ]:
print('before', train.shape)
train.dropna(subset=['salary'], inplace=True)
train.reset_index(drop=True, inplace=True)
print('after', train.shape)

In [ ]:
print('before', len(train[train.duplicated(keep=False)]))
train.drop_duplicates(inplace=True)
train.reset_index(drop=True, inplace=True)
print('after', len(train[train.duplicated(keep=False)]))

In [ ]:
concat = pd.concat([train, test], ignore_index=False)
nanconcat = (concat.isnull().sum()*100/len(concat)).to_frame()
nanconcat.columns = ['nan %']
nanconcat.loc[nanconcat['nan %'] > 0]

In [ ]:
train = train.fillna('unknown')
test = test.fillna('unknown')
concat = pd.concat([train, test], ignore_index=False)

In [ ]:
var = 'experience_level'
data = pd.concat([train['salary'], train[var]], axis=1)

f, ax = plt.subplots(figsize=(16, 8))
fig = sns.boxplot(x=var, y='salary', data=data)
fig.axis()

In [ ]:
var = 'job_benefits'
concat[var].value_counts().rename_axis(var).reset_index(name='frekuensi')

In [ ]:
var = 'company_process_time'
concat[var].value_counts().rename_axis(var).reset_index(name='frekuensi')

In [ ]:
var = 'company_size'
data = pd.concat([train['salary'], train[var]], axis=1)

f, ax = plt.subplots(figsize=(16, 8))
fig = sns.boxplot(x=var, y='salary', data=data)
fig.axis()

In [ ]:
var = 'company_industry'
df = concat[var].value_counts().rename_axis(var).reset_index(name='frekuensi')

In [ ]:
var = 'job_title'
concat[var].value_counts().rename_axis(var).reset_index(name='frekuensi')

In [ ]:
var = 'location'
concat[var].value_counts().rename_axis(var).reset_index(name='frekuensi')

In [ ]:
var = 'salary_currency'
concat[var].value_counts().rename_axis(var).reset_index(name='frekuensi')

In [ ]:
var = 'career_level'
data = pd.concat([train['salary'], train[var]], axis=1)

f, ax = plt.subplots(figsize=(16, 8))
fig = sns.boxplot(x=var, y='salary', data=data)
fig.axis()

In [ ]:
var = 'education_level'
concat[var].value_counts().rename_axis(var).reset_index(name='frekuensi')

In [ ]:
concat['education_level'] = concat['education_level'].str.split(',').str[0]

In [ ]:
var = 'education_level'
concat[var].value_counts().rename_axis(var).reset_index(name='frekuensi')

In [ ]:
var = 'employment_type'
concat[var].value_counts().rename_axis(var).reset_index(name='frekuensi')

In [ ]:
var = 'job_function'
concat[var].value_counts().rename_axis(var).reset_index(name='frekuensi')

In [ ]:
var = 'job_description'
concat[var].value_counts().rename_axis(var).reset_index(name='frekuensi')

In [ ]:
concat.drop('job_title', axis=1, inplace=True)
concat.replace(['kota banda aceh','langsa','lhokseumawe',
                'sabang','subulussalam','meulaboh'], 'aceh', inplace=True)
concat.replace(['pangkal pinang', 'bangka','belitung'], 'bangka belitung', inplace=True)
concat.replace(['sungai penuh'], 'jambi', inplace=True)
concat.replace(['batam','tanjung pinang','bintan'], 'kepulauan riau', inplace=True)
concat.replace(['bandar lampung','metro'], 'lampung', inplace=True)
concat.replace(['dumai','pekanbaru'], 'riau', inplace=True)
concat.replace(['bukittinggi','padang','padang panjang','pariaman',
                'payakumbuh','sawahlunto','solok'], 'sumatera barat', inplace=True)
concat.replace(['lubuklinggau','pagar alam','palembang',
                'prabumulih','muara enim'], 'sumatera selatan', inplace=True)
concat.replace(['binjai','gunungsitoli','medan','padang sidempuan',
                'pematangsiantar','sibolga','tanjungbalai','tebing tinggi',
                'asahan kisaran','nias','tapanuli','tanjung balai'], 'sumatera utara', inplace=True)
concat.replace(['cilegon','serang','tangerang selatan','tangerang',
                'rangkasbitung','pandeglang','cikupa'], 'banten', inplace=True)
concat.replace(['jakarta barat','jakarta selatan','jakarta timur',
                'jakarta utara', 'jakarta pusat', 'kepulauan seribu'], 'jakarta raya', inplace=True)
concat.replace(['bandung','bekasi','bogor','cimahi','cirebon','depok',
                'sukabumi','tasikmalaya','banjar','cikarang','karawang',
                'purwakarta','cileungsi','cianjur','citeureup','cibinong',
                'pangandaran'], 'jawa barat', inplace=True)
concat.replace(['magelang','pekalongan','salatiga','semarang','surakarta',
                'tegal','sleman','purwokerto','bantul','kudus','jepara','klaten',
                'purworejo','cilacap','brebes','ungaran','demak','banjarnegara','pemalang',
                'purbalingga','wonogiri '], 'jawa tengah', inplace=True)
concat.replace(['kulon progo'], 'yogyakarta', inplace=True)
concat.replace(['batu','blitar','kediri','madiun','malang',
                'mojokerto','pasuruan','probolinggo','surabaya',
                'sidoarjo','gresik','jember','banyuwangi','ponorogo',
                'madura','tuban','sampang'], 'jawa timur', inplace=True)
concat.replace(['pontianak','singkawang','sintang',
                'ketapang','mempawah','sambas'], 'kalimantan barat', inplace=True)
concat.replace(['banjarbaru','banjarmasin','tanah bumbu','balangan',
                'hulu sungai tengah','kotabaru','tanah laut',
                'barito kuala','tabalong'], 'kalimantan selatan', inplace=True)
concat.replace(['palangkaraya','kapuas','barito utara',
                'gunung mas','kotawaringin timur',
                'kotawaringin barat','murung raya'], 'kalimantan tengah', inplace=True)
concat.replace(['balikpapan','bontang','samarinda',
                'paser','kutai timur',
                'kutai barat','kutai kartanegara',
                'mahakam ulu','penajam paser utara'], 'kalimantan timur', inplace=True)
concat.replace(['tarakan','nunukan','bulungan','malinau'], 'kalimantan utara', inplace=True)
concat.replace(['ternate','halmahera'], 'maluku utara', inplace=True)
concat.replace(['ambon','maluku barat'], 'maluku', inplace=True)
concat.replace(['sorong','manokwari', 'papua barat', 'papua', 
                'jayapura','timika','merauke'], 'pulau papua', inplace=True)
concat.replace(['denpasar','badung','kuta','gianyar','seminyak',
                'klungkung','ubud','karangasem'], 'bali', inplace=True)
concat.replace(['bima','mataram','lombok','sumbawa'], 'nusa tenggara barat', inplace=True)
concat.replace(['kupang'], 'nusa tenggara timur', inplace=True)
concat.replace(['makassar','palopo','pare-pare','maros',
                'gowa','bone','toraja'], 'sulawesi selatan', inplace=True)
concat.replace(['palu','poso'], 'sulawesi tengah', inplace=True)
concat.replace(['baubau','kendari'], 'sulawesi tenggara', inplace=True)
concat.replace(['bitung','kotamobagu','manado','tomohon',
                'minahasa'], 'sulawesi utara', inplace=True)
concat.replace(['mamuju'], 'sulawesi barat', inplace=True)
concat.drop(concat[concat['salary_currency'] == 'usd'].index, inplace = True)
concat.reset_index(drop=True,inplace=True)
concat.drop('salary_currency', axis=1, inplace=True)
concat['experience_level'].replace('unknown', 'kurang dari 1 tahun', inplace=True)
concat.drop(concat[concat['education_level'] == 'doktor (s3)'].index, inplace = True)
concat.reset_index(drop=True, inplace=True)
concat['employment_type'] = concat['employment_type'].str.split(',').str[0]
concat['job_function'] = concat['job_function'].str.split(',').str[0]
concat.drop('job_benefits', axis=1, inplace=True)
concat['company_process_time'].replace('unknown', 'lebih dari 1 bulan', inplace=True)
concat['company_size'].replace('unknown', '1- 50 pekerja', inplace=True)
concat['company_industry'].replace('unknown', 'lainnya', inplace=True)
var = 'company_industry'
counts = concat[var].value_counts().rename_axis(var).reset_index(name='counts')
for i in range(len(concat)):
    if concat.loc[i, var] == 'call center/it-enabled services/bpo':
        concat.loc[i, var] = counts.loc[7, var]
    elif concat.loc[i, var] == 'kayu/fiber/kertas':
        concat.loc[i, var] = counts.loc[7, var]
    elif concat.loc[i, var] == 'hukum/legal':
        concat.loc[i, var] = counts.loc[7, var]
    elif concat.loc[i, var] == 'layanan umum/tenaga penggerak':
        concat.loc[i, var] = counts.loc[7, var]
    elif concat.loc[i, var] == 'keamanan/penegak hukum':
        concat.loc[i, var] = counts.loc[7, var]
    elif concat.loc[i, var] == 'pemerintahan/pertahanan':
        concat.loc[i, var] = counts.loc[7, var]
    elif concat.loc[i, var] == 'tembakau':
        concat.loc[i, var] = counts.loc[26, var]
    elif concat.loc[i, var] == 'r&d':
        concat.loc[i, var] = counts.loc[41, var]
    elif concat.loc[i, var] == 'jurnalisme':
        concat.loc[i, var] = counts.loc[34, var]
    elif concat.loc[i, var] == 'luar angkasa/aviasi/pesawat terbang':
        concat.loc[i, var] = counts.loc[12, var]
    elif concat.loc[i, var] == 'jual beli saham/sekuritas':
        concat.loc[i, var] = counts.loc[6, var]     
    elif concat.loc[i, var] == 'olahraga':
        concat.loc[i, var] = counts.loc[11, var]  
    elif concat.loc[i, var] == 'pemerintahan/pertahanan':
        concat.loc[i, var] = counts.loc[50, var] 
    elif concat.loc[i, var] == 'kelautan/aquakultur':
        concat.loc[i, var] = counts.loc[26, var] 
    elif concat.loc[i, var] == 'travel/pariwisata':
        concat.loc[i, var] = counts.loc[35, var]
    elif concat.loc[i, var] == 'permata/perhiasan':
        concat.loc[i, var] = counts.loc[29, var]   
    elif concat.loc[i, var] == 'lingkungan/kesehatan/keamanan':
        concat.loc[i, var] = counts.loc[11, var]   
    elif concat.loc[i, var] == 'pakaian':
        concat.loc[i, var] = counts.loc[28, var]  
    elif concat.loc[i, var] == 'minyak/gas/petroleum':
        concat.loc[i, var] = counts.loc[27, var]
concat.drop('job_description', axis=1, inplace=True)

In [ ]:
concat['job_position'] = concat['career_level'] + ' ' + concat['job_function']
var = 'job_position'
concat[var].value_counts().rename_axis(var).reset_index(name='frekuensi')

In [ ]:
enc = OrdinalEncoder()
catcol = [col for col in concat.columns if col != 'id' and col != 'salary']
concat[catcol] = enc.fit_transform(concat[catcol])

In [ ]:
train = concat[~concat.salary.isnull()].reset_index(drop=True)
test = concat[concat.salary.isnull()].reset_index(drop=True)

train.drop('id', axis=1, inplace=True)
test.drop('salary', axis=1, inplace=True)
test = test.astype('int32')

X = train.drop('salary', axis=1)
y = train['salary']

In [ ]:
train[train.duplicated([col for col in train.columns if col != 'salary'], keep=False)]

In [ ]:
train = train.groupby([col for col in train.columns if col != 'salary'], as_index=True)['salary'].mean().reset_index()

In [ ]:
train[train.duplicated([col for col in train.columns if col != 'salary'], keep=False)]

In [ ]:
sns.distplot(train['salary'])

In [ ]:
train['salary'].describe()

In [ ]:
train.loc[train['salary'] < 500000]

In [ ]:
train.drop(train[train['salary'] < 500000].index, inplace = True)

In [ ]:
X = train.drop('salary', axis=1)
y = train['salary']

In [ ]:
estimator = CatBoostRegressor(logging_level='Silent')
model = estimator.fit(X,y)
importance = model.feature_importances_
feature = X.columns
sort = importance.argsort()[::-1][:len(importance)]
fig = plt.gcf()
plt.barh(feature[importance.argsort()[::1][:len(importance)]], importance[importance.argsort()[::1][:len(importance)]])

In [ ]:
train = train.groupby([col for col in train.columns if col != 'salary'], as_index=True)['salary'].mean().reset_index()

In [ ]:
X = train.drop('salary', axis=1)
y = train['salary']

In [ ]:
cv = KFold(n_splits=10, shuffle=True, random_state=2022)

dtr = DecisionTreeRegressor()
rfr = RandomForestRegressor()
xgb = XGBRegressor()
ctb = CatBoostRegressor(logging_level='Silent')

In [ ]:
np.sqrt(-cross_val_score(dtr, X, y, cv=cv, scoring='neg_mean_squared_error', n_jobs=-1)).mean()

In [ ]:
np.sqrt(-cross_val_score(rfr, X, y, cv=cv, scoring='neg_mean_squared_error', n_jobs=-1)).mean()

In [ ]:
np.sqrt(-cross_val_score(xgb, X, y, cv=cv, scoring='neg_mean_squared_error', n_jobs=-1)).mean()

In [ ]:
np.sqrt(-cross_val_score(ctb, X, y, cv=cv, scoring='neg_mean_squared_error', n_jobs=-1)).mean()

In [ ]:
train.drop('employment_type', axis=1, inplace=True)
train = train.groupby([col for col in train.columns if col != 'salary'], as_index=True)['salary'].mean().reset_index()
test.drop('employment_type', axis=1, inplace=True)

In [ ]:
X = train.drop('salary', axis=1)
y = train['salary']

In [ ]:
np.sqrt(-cross_val_score(ctb, X, y, cv=cv, scoring='neg_mean_squared_error', n_jobs=-1)).mean()

In [ ]:
def xgb_obj(params):
    params = {
        'max_bin' : int(params['max_bin']),
        'max_depth' : int(params['max_depth']),
        'max_leaves' : int(params['max_leaves']),
        'learning_rate' : '{:.6f}'.format(params['learning_rate']),
        'gamma' : '{:.6f}'.format(params['gamma']),
        'reg_alpha' : '{:.6f}'.format(params['reg_alpha']),
        'reg_lambda' : '{:.6f}'.format(params['reg_lambda']),
        'min_child_weight' : '{:.6f}'.format(params['min_child_weight']),
        'colsample_bytree' : '{:.6f}'.format(params['colsample_bytree']),
        'colsample_bynode' :'{:.6f}'.format(params['colsample_bynode']),
        'colsample_bylevel' : '{:.6f}'.format(params['colsample_bylevel']),
        'subsample' : '{:.6f}'.format(params['subsample'])
    }
    model = XGBRegressor(**params)
    mse = -cross_val_score(model, X, y, cv=cv, scoring='neg_mean_squared_error', n_jobs=-1).mean()
    
    return np.sqrt(mse)

space = {
    'max_bin' : hp.quniform('max_bin', 2**3, 2**7, 1),
    'max_depth' : hp.quniform('max_depth', 2, 25, 1),
    'max_leaves' : hp.quniform('max_leaves', 32, 2**12, 1),
    'seed' : hp.quniform('seed', 16, 48, 2),
    'learning_rate' : hp.loguniform('learning_rate', np.log(0.005), np.log(0.2)),
    'gamma' : hp.uniform('gamma', 0, 5),
    'reg_alpha' : hp.uniform('reg_alpha', 0, 5),
    'reg_lambda' : hp.uniform('reg_lambda', 0, 5),
    'min_child_weight' : hp.uniform('min_child_weight', 0, 5),
    'colsample_bytree' : hp.quniform('colsample_bytree', 0.1, 1, 0.01),
    'colsample_bynode' : hp.quniform('colsample_bynode', 0.1, 1, 0.01),
    'colsample_bylevel' : hp.quniform('colsample_bylevel', 0.1, 1, 0.01),
    'subsample' : hp.quniform('subsample', 0.5, 1, 0.05)
    }

def hoptxgb():
    best = fmin(fn=xgb_obj,
            space=space, 
            algo=tpe.suggest,
            max_evals=100000)
    print('tuned_xgb = {}'.format(best))

In [ ]:
def ctb_obj(params):
    params = {
        'iterations': int(params['iterations']),
        'learning_rate': params['learning_rate'],
        'l2_leaf_reg': params['l2_leaf_reg'],
        'random_strength': params['random_strength'],
        'depth': int(params['depth'])
    }
    model = XGBRegressor(**params)
    mse = -cross_val_score(model, X, y, cv=cv, scoring='neg_mean_squared_error', n_jobs=-1).mean()
    
    return np.sqrt(mse)

space = {
    'iterations': hp.quniform('iterations', 1000, 2000),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.03),
    'l2_leaf_reg': hp.uniform('l2_leaf_reg', 3, 18),
    'random_strength': hp.uniform('random_strength', 3, 18),
    'depth': hp.quniform('depth', 6, 12, 1)
}

def hoptctb():
    best = fmin(fn=ctb_obj,
            space=space, 
            algo=tpe.suggest,
            max_evals=100000)
    print('tuned_ctb = {}'.format(best))

In [ ]:
xgb = XGBRegressor(**tuned_xgb)
ctb = CatBoostRegressor(**tuned_ctb, logging_level='Silent')
voting = VotingRegressor([('xgb', xgb), ('ctb', ctb)])

In [ ]:
np.sqrt(-cross_val_score(xgb, X, y, cv=cv, scoring='neg_mean_squared_error', n_jobs=-1)).mean()

In [ ]:
np.sqrt(-cross_val_score(ctb, X, y, cv=cv, scoring='neg_mean_squared_error', n_jobs=-1)).mean()

In [ ]:
np.sqrt(-cross_val_score(voting, X, y, cv=cv, scoring='neg_mean_squared_error', n_jobs=-1)).mean()

In [ ]:
feat = test.drop('id', axis=1)

sub = pd.DataFrame()
sub['id'] = test['id']
sub['salary'] = (voting.predict(feat))

sub.head()